In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import classification_report

In [2]:
# URL of the OwnCloud file
url = 'https://owncloud.ut.ee/owncloud/s/pMRrERiTnpE9srJ/download'

response = requests.get(url)
response.raise_for_status()  # raise an error if the download failed
zip_file = ZipFile(BytesIO(response.content))
zip_file.extractall()

csv_file_name = 'BPI_Challenge_2012_filtered.csv'
df = pd.read_csv(csv_file_name)

df

,case_id,AMOUNT_REQ,resource,activity,Start_Time,End_Time,REG_DATE
0,196554,6000,112.0,A_DECLINED,2011-12-29T21:43:20.708,2011-12-29T21:43:20.708,2011-12-29T21:42:45.706
1,196557,10000,112.0,A_SUBMITTED,2011-12-29T21:48:26.730,2011-12-29T21:48:26.730,2011-12-29T21:48:26.730
2,196557,10000,112.0,A_PARTLYSUBMITTED,2011-12-29T21:48:26.956,2011-12-29T21:48:26.956,2011-12-29T21:48:26.730
3,196557,10000,112.0,A_PREACCEPTED,2011-12-29T21:49:09.983,2011-12-29T21:49:09.983,2011-12-29T21:48:26.730
4,196557,10000,11003.0,W_Completeren aanvraag,2011-12-30T12:23:05.764,2011-12-30T12:37:06.319,2011-12-29T21:48:26.730
...,...,...,...,...,...,...,...
12474,202833,10000,112.0,A_SUBMITTED,2012-01-21T16:01:47.362,2012-01-21T16:01:47.362,2012-01-21T16:01:47.362
12475,202833,10000,112.0,A_PARTLYSUBMITTED,2012-01-21T16:01:47.906,2012-01-21T16:01:47.906,2012-01-21T16:01:47.362
12476,202833,10000,112.0,A_PREACCEPTED,2012-01-21T16:02:26.922,2012-01-21T16:02:26.922,2012-01-21T16:01:47.362
12477,202833,10000,10932.0,W_Completeren aanvraag,2012-01-21T16:33:53.726,2012-01-21T16:44:50.485,2012-01-21T16:01:47.362


In [3]:
df.dtypes

case_id         int64
AMOUNT_REQ      int64
resource      float64
activity       object
Start_Time     object
End_Time       object
REG_DATE       object
dtype: object

In [4]:
df_clean = df.copy()

df_clean['Start_Time'] = pd.to_datetime(df_clean['Start_Time'])
df_clean['End_Time'] = pd.to_datetime(df_clean['End_Time'])
df_clean['REG_DATE'] = pd.to_datetime(df_clean['REG_DATE'])

df_clean['duration'] = df_clean['End_Time'] - df_clean['Start_Time']
case_durations = df_clean.groupby('case_id')['duration'].sum()
mean_duration = case_durations.mean()
df_clean['label'] = df_clean['case_id'].apply(lambda x: 1 if case_durations[x] > mean_duration else 0)

In [5]:
df_clean['Start_month'] = df_clean['Start_Time'].dt.month
df_clean['Start_day_of_week'] = df_clean['Start_Time'].dt.weekday
df_clean['Start_seconds_rel'] = (df_clean['Start_Time'] - df_clean['Start_Time'].dt.normalize()).dt.total_seconds().astype(int)

df_clean['End_month'] = df_clean['End_Time'].dt.month
df_clean['End_day_of_week'] = df_clean['End_Time'].dt.weekday
df_clean['End_seconds_rel'] = (df_clean['End_Time'] - df_clean['End_Time'].dt.normalize()).dt.total_seconds().astype(int)

df_clean[['Start_Time', 'Start_month', 'Start_day_of_week', 'Start_seconds_rel',
          'End_Time', 'End_month', 'End_day_of_week', 'End_seconds_rel']]

,Start_Time,Start_month,Start_day_of_week,Start_seconds_rel,End_Time,End_month,End_day_of_week,End_seconds_rel
0,2011-12-29 21:43:20.708,12,3,78200,2011-12-29 21:43:20.708,12,3,78200
1,2011-12-29 21:48:26.730,12,3,78506,2011-12-29 21:48:26.730,12,3,78506
2,2011-12-29 21:48:26.956,12,3,78506,2011-12-29 21:48:26.956,12,3,78506
3,2011-12-29 21:49:09.983,12,3,78549,2011-12-29 21:49:09.983,12,3,78549
4,2011-12-30 12:23:05.764,12,4,44585,2011-12-30 12:37:06.319,12,4,45426
...,...,...,...,...,...,...,...,...
12474,2012-01-21 16:01:47.362,1,5,57707,2012-01-21 16:01:47.362,1,5,57707
12475,2012-01-21 16:01:47.906,1,5,57707,2012-01-21 16:01:47.906,1,5,57707
12476,2012-01-21 16:02:26.922,1,5,57746,2012-01-21 16:02:26.922,1,5,57746
12477,2012-01-21 16:33:53.726,1,5,59633,2012-01-21 16:44:50.485,1,5,60290


In [6]:
def time_based_train_test_split(df, test_size):
    case_id_col = 'case_id'
    timestamp_col = 'Start_Time'
    activity_col = 'activity'

    df = df.sort_values([timestamp_col, activity_col], ascending=True, kind='mergesort')

    start_timestamps = df.groupby(case_id_col)[timestamp_col].min().reset_index()
    start_timestamps = start_timestamps.sort_values(timestamp_col, ascending=True, kind='mergesort')

    num_train = int((1 - test_size) * len(start_timestamps))
    train_cases = start_timestamps[case_id_col].head(num_train)
    split_ts = start_timestamps[timestamp_col].iloc[num_train]

    train_df = df[df[case_id_col].isin(train_cases) & (df[timestamp_col] < split_ts)]
    test_df = df[~df[case_id_col].isin(train_cases)]

    return train_df, test_df

In [7]:
train_df, test_df = time_based_train_test_split(df_clean, 0.2)

In [8]:
len(train_df)

9552

In [9]:
len(test_df)

1511

In [10]:
def generate_prefix_data(data, min_length, max_length, gap=1):
    # generate prefix data (each possible prefix becomes a trace)
    data['case_length'] = data.groupby('case_id')['activity'].transform(len)

    dt_prefixes = data[data['case_length'] >= min_length].groupby('case_id').head(min_length)
    dt_prefixes["prefix_nr"] = 1
    dt_prefixes["orig_case_id"] = dt_prefixes['case_id']
    for nr_events in range(min_length+gap, max_length+1, gap):
        tmp = data[data['case_length'] >= nr_events].groupby('case_id').head(nr_events)
        tmp["orig_case_id"] = tmp['case_id']
        tmp['case_id'] = tmp['case_id'].apply(lambda x: "%s_%s"%(x, nr_events))
        tmp["prefix_nr"] = nr_events
        dt_prefixes = pd.concat([dt_prefixes, tmp], axis=0)

    dt_prefixes['case_length'] = dt_prefixes['case_length'].apply(lambda x: min(max_length, x))

    return dt_prefixes

In [11]:
df_train_prefixes = generate_prefix_data(train_df, 1, 52)
df_test_prefixes = generate_prefix_data(test_df, 1, 52)

In [12]:
len(df_train_prefixes)

66988

In [13]:
len(df_test_prefixes)

5634

In [14]:
def preprocess(df, case_id_col, first_cols, last_cols, cat_cols, fillna=True, columns=None):
    first_data = df.groupby(case_id_col).first()[first_cols]
    last_data = df.groupby(case_id_col).last()[last_cols]

    if cat_cols:
        last_data = pd.get_dummies(last_data, columns=cat_cols)

    combined_data = pd.concat([first_data, last_data], axis=1)

    if fillna:
        combined_data = combined_data.fillna(0)

    if columns is not None:
        missing_cols = [col for col in columns if col not in combined_data.columns]
        for col in missing_cols:
            combined_data[col] = 0
        combined_data = combined_data[columns]
    else:
        columns = combined_data.columns

    return combined_data.reset_index(), columns

In [15]:
train_df_preprocessed, used_columns = preprocess(
    df=df_train_prefixes,
    case_id_col='case_id',
    first_cols=['AMOUNT_REQ', 'label'],
    last_cols=['Start_month', 'Start_day_of_week', 'Start_seconds_rel',
               'End_month', 'End_day_of_week', 'End_seconds_rel', 'resource', 'activity'],
    cat_cols=['resource', 'activity'],
    fillna=True
)
test_df_preprocessed, _ = preprocess(
    df=df_test_prefixes,
    case_id_col='case_id',
    first_cols=['AMOUNT_REQ', 'label'],
    last_cols=['Start_month', 'Start_day_of_week', 'Start_seconds_rel',
               'End_month', 'End_day_of_week', 'End_seconds_rel', 'resource', 'activity'],
    cat_cols=['resource', 'activity'],
    fillna=True,
    columns=used_columns
)

In [16]:
model = XGBClassifier(n_estimators=1000, random_state=0)
model.fit(train_df_preprocessed.drop(columns=['case_id', 'label']), train_df_preprocessed['label'])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=0, ...)

In [17]:
preds_pos_label_idx = np.where(model.classes_ == 1)[0][0]
preds = model.predict_proba(test_df_preprocessed.drop(columns=['case_id', 'label']))[:,preds_pos_label_idx]
score = roc_auc_score(test_df_preprocessed['label'], preds)
score

0.8236138290932813

In [18]:
print(classification_report(test_df_preprocessed['label'], model.predict(test_df_preprocessed.drop(columns=['case_id', 'label']))))

              precision    recall  f1-score   support

           0       0.96      0.88      0.92      1365
           1       0.36      0.66      0.47       146

    accuracy                           0.86      1511
   macro avg       0.66      0.77      0.69      1511
weighted avg       0.90      0.86      0.87      1511



In [19]:
train_df_preprocessed, used_columns = preprocess(
    df=df_train_prefixes,
    case_id_col='case_id',
    first_cols=['AMOUNT_REQ', 'label'],
    last_cols=['resource', 'activity'],
    cat_cols=['resource', 'activity'],
    fillna=True
)
test_df_preprocessed, _ = preprocess(
    df=df_test_prefixes,
    case_id_col='case_id',
    first_cols=['AMOUNT_REQ', 'label'],
    last_cols=['resource', 'activity'],
    cat_cols=['resource', 'activity'],
    fillna=True,
    columns=used_columns
)

In [20]:
model = XGBClassifier(n_estimators=1000, random_state=0)
model.fit(train_df_preprocessed.drop(columns=['case_id', 'label']), train_df_preprocessed['label'])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=0, ...)

In [21]:
preds_pos_label_idx = np.where(model.classes_ == 1)[0][0]
preds = model.predict_proba(test_df_preprocessed.drop(columns=['case_id', 'label']))[:,preds_pos_label_idx]
score = roc_auc_score(test_df_preprocessed['label'], preds)
score

0.8049726529178584

In [22]:
print(classification_report(test_df_preprocessed['label'], model.predict(test_df_preprocessed.drop(columns=['case_id', 'label']))))

              precision    recall  f1-score   support

           0       0.96      0.89      0.92      1365
           1       0.38      0.63      0.47       146

    accuracy                           0.86      1511
   macro avg       0.67      0.76      0.70      1511
weighted avg       0.90      0.86      0.88      1511

